<a href="https://colab.research.google.com/github/Andru-1987/74235-_DataScience_I/blob/main/clase_9/clase_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Asistentes de voces
* `pyttsx3`: is suitable for offline applications or when you need quick, local speech synthesis.
* `gTTS`: is ideal when you need high-quality, natural-sounding voices and have an internet connection.

In [42]:
!pip install pyttsx3 gTTS beautifulsoup4 requests pillow pytesseract

In [43]:
import pyttsx3
import pytesseract
import requests
from gtts import gTTS
from PIL import Image
from bs4 import BeautifulSoup
import pygame
import io
import os
from abc import ABC, abstractmethod
from typing import Optional, Protocol


from IPython.display import Audio, display


In [44]:
# Interfaces/Protocolos para inyección de dependencias
class TextToSpeechInterface(Protocol):
    """Interfaz para servicios de texto a voz"""
    def speak(self, text: str) -> None:
        """Texto a voz"""
        ...

class OCRInterface(Protocol):
    """Interfaz para servicios de OCR"""
    def extract_text(self, image_path: str) -> str:
        """Texto a imagen"""
        ...

class WebScrapingInterface(Protocol):
    """Interfaz para servicios de web scraping"""
    def extract_content(self, url: str) -> str:
        """Toma contenido web"""
        ...


In [45]:
import sys
import io
from gtts import gTTS
from IPython.display import Audio, display

class PyttxGTTSTextToSpeech:
    """
    Implementación compatible con Google Colab.
    Si se detecta que está en Colab, usa gTTS + Audio de IPython.
    Si está en entorno local, intenta usar pyttsx3 (si está disponible).
    """

    def __init__(self, rate: int = 150, voice_index: int = 0, language: str = 'es'):
        self.language = language
        self.in_colab = 'google.colab' in sys.modules

        if not self.in_colab:
            try:
                import pyttsx3
                self.engine = pyttsx3.init()
                self.engine.setProperty("rate", rate)

                voices = self.engine.getProperty('voices')
                if voices and len(voices) > voice_index:
                    self.engine.setProperty('voice', voices[voice_index].id)
            except Exception as e:
                print(f"No se pudo inicializar pyttsx3: {e}")
                self.engine = None

    def speak(self, text: str) -> None:
        """Convierte texto a voz, compatible con Google Colab y entorno local."""
        try:
            if self.in_colab:
                # En Google Colab: usar gTTS
                tts = gTTS(text=text, lang=self.language)
                audio_buffer = io.BytesIO()
                tts.write_to_fp(audio_buffer)
                audio_buffer.seek(0)
                display(Audio(audio_buffer.read(), autoplay=True))
            elif hasattr(self, 'engine') and self.engine:
                # En entorno local con pyttsx3
                self.engine.say(text)
                self.engine.runAndWait()
            else:
                raise RuntimeError("No hay motor TTS disponible.")
        except Exception as e:
            print(f"Error en TTS: {e}")


class BeautifulSoupWebScraper:
    """Implementación de web scraping usando BeautifulSoup"""

    def __init__(self, timeout: int = 10, max_paragraphs: int = 3):
        self.timeout = timeout
        self.max_paragraphs = max_paragraphs
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        }

    def extract_content(self, url: str) -> str:
        """Extrae contenido de una página web"""
        try:
            response = requests.get(url, headers=self.headers, timeout=self.timeout)
            response.raise_for_status()

            soup = BeautifulSoup(response.content, 'html.parser')

            # Extraer párrafos
            paragraphs = soup.find_all('p')
            text_content = []

            for p in paragraphs[:self.max_paragraphs]:
                text = p.get_text(strip=True)
                if text:  # Solo agregar párrafos no vacíos
                    text_content.append(text)

            return ' '.join(text_content)

        except requests.RequestException as e:
            raise Exception(f"Error al conectar con la URL: {e}")
        except Exception as e:
            raise Exception(f"Error al procesar el contenido: {e}")


# Clase principal del asistente con inyección de dependencias
class AsistenteVirtual:
    """Asistente virtual que utiliza inyección de dependencias"""

    def __init__(
        self,
        tts_service: PyttxGTTSTextToSpeech,
        ocr_service: Optional[OCRInterface] = None,
        web_scraper: Optional[WebScrapingInterface] = None
    ):
        self.tts_service = tts_service
        self.ocr_service = ocr_service
        self.web_scraper = web_scraper

    def hablar(self, texto: str) -> None:
        """Hace que el asistente hable"""
        print("Asistente:", texto)
        self.tts_service.speak(texto)

    def texto_a_voz(self, texto: str) -> None:
        """Convierte texto a voz"""
        self.hablar(texto)

    def imagen_a_texto(self, path_imagen: str) -> None:
        """Extrae texto de una imagen y lo lee en voz alta"""
        if not self.ocr_service:
            self.hablar("Servicio OCR no disponible")
            return

        try:
            texto = self.ocr_service.extract_text(path_imagen)
            if texto:
                self.hablar("Texto extraído:")
                print(f"Texto extraído: {texto}")
                self.hablar(texto)
            else:
                self.hablar("No se pudo extraer texto de la imagen")
        except Exception as e:
            self.hablar("No pude procesar la imagen")
            print(f"Error: {e}")

    def extraer_info_web(self, url: str) -> None:
        """Extrae información de una página web y la lee"""
        if not self.web_scraper:
            self.hablar("Servicio de web scraping no disponible")
            return

        try:
            contenido = self.web_scraper.extract_content(url)
            if contenido:
                self.hablar("Información de la página:")
                print(f"Contenido extraído: {contenido}")
                self.hablar(contenido)
            else:
                self.hablar("No se pudo extraer contenido de la página")
        except Exception as e:
            self.hablar("Error al conectarme a la web")
            print(f"Error: {e}")




### Clase especial para Imagen a Texto

In [46]:
! lsb_release -a

No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 22.04.4 LTS
Release:	22.04
Codename:	jammy


In [47]:
# Ubuntu/Debian
!sudo apt-get install tesseract-ocr tesseract-ocr-spa tesseract-ocr-eng espeak-ng

# # CentOS/RHEL
# !sudo yum install tesseract tesseract-langpack-spa tesseract-langpack-eng

# # Arch Linux
# !sudo pacman -S tesseract tesseract-data-spa tesseract-data-eng

# # Fedora
# !sudo dnf install tesseract tesseract-langpack-spa tesseract-langpack-eng

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
tesseract-ocr-eng is already the newest version (1:4.00~git30-7274cfa-1.1).
tesseract-ocr-spa is already the newest version (1:4.00~git30-7274cfa-1.1).
espeak-ng is already the newest version (1.50+dfsg-10ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [48]:
class TesseractOCR:
    """Implementación de OCR usando Tesseract"""

    def __init__(self, language: str = 'spa+eng', config: str = ''):
        self.language = language
        self.config = config

        # Configurar Tesseract según el sistema operativo
        self._configure_tesseract()

        # Verificar que Tesseract está instalado
        self._verify_tesseract_installation()

    def _configure_tesseract(self):
        """Configura la ruta de Tesseract según el sistema operativo"""
        import platform
        import shutil

        system = platform.system().lower()

        if system == 'linux':
            # En Linux, buscar en rutas comunes
            possible_paths = [
                '/usr/bin/tesseract',
                '/usr/local/bin/tesseract',
                '/opt/tesseract/bin/tesseract',
                shutil.which('tesseract')  # Buscar en PATH
            ]

            for path in possible_paths:
                if path and os.path.exists(path):
                    pytesseract.pytesseract.tesseract_cmd = path
                    print(f"Tesseract encontrado en: {path}")
                    return

            # Si no se encuentra, usar el comando por defecto
            pytesseract.pytesseract.tesseract_cmd = 'tesseract'

        elif system == 'windows':
            # Configuración para Windows
            possible_paths = [
                r'C:\Program Files\Tesseract-OCR\tesseract.exe',
                r'C:\Program Files (x86)\Tesseract-OCR\tesseract.exe',
                r'C:\tesseract\tesseract.exe'
            ]

            for path in possible_paths:
                if os.path.exists(path):
                    pytesseract.pytesseract.tesseract_cmd = path
                    print(f"Tesseract encontrado en: {path}")
                    return

            # Si no se encuentra, usar el comando por defecto
            pytesseract.pytesseract.tesseract_cmd = 'tesseract'

        elif system == 'darwin':  # macOS
            # En macOS con Homebrew
            possible_paths = [
                '/opt/homebrew/bin/tesseract',  # Apple Silicon
                '/usr/local/bin/tesseract',     # Intel
                shutil.which('tesseract')
            ]

            for path in possible_paths:
                if path and os.path.exists(path):
                    pytesseract.pytesseract.tesseract_cmd = path
                    print(f"Tesseract encontrado en: {path}")
                    return

            pytesseract.pytesseract.tesseract_cmd = 'tesseract'

    def _verify_tesseract_installation(self):
        """Verifica que Tesseract esté instalado y disponible"""
        try:
            version = pytesseract.get_tesseract_version()
            print(f"Tesseract version: {version}")

            # Verificar idiomas disponibles
            try:
                languages = pytesseract.get_languages()
                print(f"Idiomas disponibles: {languages}")

                # Verificar si los idiomas solicitados están disponibles
                requested_langs = self.language.split('+')
                missing_langs = [lang for lang in requested_langs if lang not in languages]

                if missing_langs:
                    print(f"⚠️  Advertencia: Idiomas no encontrados: {missing_langs}")
                    print("Para instalar idiomas en Linux:")
                    print("Ubuntu/Debian: sudo apt-get install tesseract-ocr-spa tesseract-ocr-eng")
                    print("CentOS/RHEL: sudo yum install tesseract-langpack-spa tesseract-langpack-eng")
                    print("Arch Linux: sudo pacman -S tesseract-data-spa tesseract-data-eng")

            except Exception as e:
                print(f"No se pudieron obtener los idiomas: {e}")

        except Exception as e:
            print(f"⚠️  Error: Tesseract no está instalado o no se encuentra en PATH")
            print("Para instalar Tesseract en Linux:")
            print("Ubuntu/Debian: sudo apt-get install tesseract-ocr tesseract-ocr-spa tesseract-ocr-eng")
            print("CentOS/RHEL: sudo yum install tesseract tesseract-langpack-spa tesseract-langpack-eng")
            print("Arch Linux: sudo pacman -S tesseract tesseract-data-spa tesseract-data-eng")
            print("Fedora: sudo dnf install tesseract tesseract-langpack-spa tesseract-langpack-eng")
            raise Exception(f"Tesseract no está disponible: {e}")

    def extract_text(self, image_path: str) -> str:
        """Extrae texto de una imagen usando Tesseract"""
        try:
            # Verificar que el archivo existe
            if not os.path.exists(image_path):
                raise FileNotFoundError(f"No se encontró la imagen: {image_path}")

            # Abrir y procesar imagen
            with Image.open(image_path) as img:
                # Convertir a RGB si es necesario
                if img.mode != 'RGB':
                    img = img.convert('RGB')

                # Aplicar mejoras de imagen si es necesario
                img = self._preprocess_image(img)

                # Extraer texto con configuración mejorada
                custom_config = self._build_config()
                text = pytesseract.image_to_string(
                    img,
                    lang=self.language,
                    config=custom_config
                )

                return text.strip()

        except FileNotFoundError as e:
            raise Exception(f"No se pudo encontrar la imagen: {image_path}")
        except Exception as e:
            raise Exception(f"Error al procesar la imagen: {e}")

    def _preprocess_image(self, img: Image.Image) -> Image.Image:
        """Preprocesa la imagen para mejorar la precisión del OCR"""
        # Redimensionar si es muy pequeña
        width, height = img.size
        if width < 300 or height < 300:
            scale_factor = max(300/width, 300/height)
            new_width = int(width * scale_factor)
            new_height = int(height * scale_factor)
            img = img.resize((new_width, new_height), Image.Resampling.LANCZOS)

        return img

    def _build_config(self) -> str:
        """Construye la configuración de Tesseract optimizada"""
        base_config = '--oem 3 --psm 6'  # OCR Engine Mode 3, Page Segmentation Mode 6

        if self.config:
            return f"{base_config} {self.config}"
        else:
            return base_config

    def get_available_languages(self) -> list:
        """Obtiene la lista de idiomas disponibles"""
        try:
            return pytesseract.get_languages()
        except Exception as e:
            print(f"Error al obtener idiomas: {e}")
            return []

    def extract_text_with_confidence(self, image_path: str) -> dict:
        """Extrae texto con información de confianza"""
        try:
            with Image.open(image_path) as img:
                if img.mode != 'RGB':
                    img = img.convert('RGB')

                img = self._preprocess_image(img)

                # Obtener datos detallados
                data = pytesseract.image_to_data(
                    img,
                    lang=self.language,
                    config=self._build_config(),
                    output_type=pytesseract.Output.DICT
                )

                # Calcular confianza promedio
                confidences = [int(conf) for conf in data['conf'] if int(conf) > 0]
                avg_confidence = sum(confidences) / len(confidences) if confidences else 0

                # Extraer texto
                text = ' '.join([word for word in data['text'] if word.strip()])

                return {
                    'text': text,
                    'confidence': avg_confidence,
                    'word_count': len([word for word in data['text'] if word.strip()])
                }

        except Exception as e:
            raise Exception(f"Error al procesar imagen con confianza: {e}")


## Asistente de Voz

In [49]:
asistente_voz = AsistenteVirtual(tts_service=PyttxGTTSTextToSpeech())
asistente_voz.hablar("Ejecutando desde google colab")

Asistente: Hola, soy tu asistente de voz
